In [2]:
from ortools.sat.python import cp_model

# Tasks
- Duration
- Hard start and end time?
- Concurrent?
- Precedence
- Sentiment (first, mid, end)
- Deadline slot

In [156]:
# CONSTANTS

WORKING_HOURS = 8
SUBDIVISIONS = 4
DAYS = 5
NUM_SLOTS = WORKING_HOURS * SUBDIVISIONS * DAYS
DEADLINE_PENALTY_CONSTANT = 4
SENTIMENT_PENALTY_CONSTANT = 10



In [157]:
# PRIORITY FUNCTION
def deadline_penalty(end, deadline):
    return DEADLINE_PENALTY_CONSTANT * (NUM_SLOTS - (deadline - end))


In [158]:
class Task:
    def __init__(self, duration, start_time, concurrent, precedes, sentiment, deadline, name):
        self.duration = duration
        # OPTIONAL
        self.start_time = start_time
        self.concurrent = concurrent
        # INDEX IN ARRAY
        self.precedes = precedes
        self.sentiment = sentiment
        self.deadline = deadline
        self.name = name

In [159]:
class TaskScheduler:
    def __init__(self, tasks):
        self.tasks = tasks

In [160]:
def create_interval_variables(self):
    model = self.model
    self.schedule = []
    self.demands = []
    for t in self.tasks:
        start, end, interval = None, None, None
        if t.start_time == None:
            start = model.NewIntVar(0, NUM_SLOTS - 1, f'Task {t.name} start')
            end = model.NewIntVar(0, NUM_SLOTS - 1, f'Task {t.name} end')
        else:
            start = model.NewIntVar(t.start_time, t.start_time, f'Task {t.name} start')
            end = model.NewIntVar(t.start_time + t.duration, t.start_time + t.duration, f'Task {t.name} end')
        interval = model.NewIntervalVar(start, t.duration, end, f'Task {t.name} interval')
        interval.start = start
        interval.end = end
        self.demands.append(0 if t.concurrent else 1)
        self.schedule.append(interval)
TaskScheduler.create_interval_variables = create_interval_variables

In [161]:
def create_overlapping_constraints(self):
    model = self.model
    model.AddCumulative(self.schedule, self.demands, 1)
TaskScheduler.create_overlapping_constraints = create_overlapping_constraints

In [221]:
def create_precedence_constraints(self):
    for i, t in enumerate(self.tasks):
        for succ in t.precedes:
            self.model.Add(self.schedule[i].end <= self.schedule[succ].start)
TaskScheduler.create_precedence_constraints = create_precedence_constraints

In [222]:
def create_scheduling_penalties(self):
    self.penalty_sq = self.model.NewIntVar(0, 200000000, 'Scheduling Penalties 1/2')
    self.penalty = self.model.NewIntVar(0, 200000000000000, 'Scheulding Penalties')
    cumul = 0
    for i, s in enumerate(self.schedule):
        cumul += deadline_penalty(s.end, self.tasks[i].deadline)
        ##### TODO ###### Add sentiment penalties
        
#     self.model.Add(self.penalty == cumul)
    self.model.Add(self.penalty_sq == cumul)
    self.model.AddMultiplicationEquality(self.penalty, [self.penalty_sq, self.penalty_sq])

TaskScheduler.create_scheduling_penalties = create_scheduling_penalties    

In [223]:
def solve_model(self):
    self.model = cp_model.CpModel()
    self.solver = cp_model.CpSolver()
    
    self.create_interval_variables()
    self.create_overlapping_constraints()
    self.create_precedence_constraints()
    self.create_scheduling_penalties()
    self.model.Minimize(self.penalty)
    
    self.solver.parameters.num_search_workers = 4
    if self.solver.Solve(self.model) in [cp_model.FEASIBLE, cp_model.OPTIMAL]:
        return [(self.solver.Value(s.start), self.solver.Value(s.end)) for s in self.schedule]
TaskScheduler.solve_model = solve_model

In [238]:
scheduler = TaskScheduler([Task(1, 0, True, [], None, 160, "Coffee"), Task(10, None, False, [], None, 60, "Task1"), Task(3, None, False, [], None, 140, "Task2")])

In [239]:
scheduler.solve_model()

[(0, 1), (3, 13), (0, 3)]

# Examples

- 